# Data Pull

In [7]:
pip install ccxt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ccxt
import time
import pandas as pd
from datetime import datetime, timezone
from tqdm import tqdm
import os

## Coin Universe

In [2]:
SPOT_COLUMNS = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume',
                'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

binance_exchange = ccxt.binanceus({
    'timeout': 15000,
    'enableRateLimit': True
    # 'options': {'defaultType': 'future'}
})
markets = binance_exchange.load_markets()
all_tickers = list(markets.keys())
usdt_tickers = [ticker for ticker in all_tickers if ticker.endswith('USDT')]
active_usdt_tickers = [
    ticker for ticker in usdt_tickers if markets[ticker]['active']
]
len(active_usdt_tickers)

162

In [3]:
def get_spot(exchange, symbol, interval = '1d',
                    startTime = None,
                    endTime = None,
                    limit = 1000):

    if (startTime == None and endTime == None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'limit': limit})
    elif (startTime == None and endTime != None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'endTime': endTime,
                                        'limit': limit})
    elif (startTime != None and endTime == None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'startTime': startTime,
                                        'limit': limit})
    else:
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'limit': limit})

def convert_to_seconds(time_input):
    number = int(time_input[:-1])
    unit = time_input[-1]

    if unit == 's':
        return number
    elif unit == 'm':
        return number * 60
    elif unit == 'h':
        return number * 3600
    elif unit == 'd':
        return number * 86400
    else:
        raise ValueError("Unsupported time unit")

def transform_timestamp(timestamp_integer):
    '''
    As data points involved milliseconds, we need to transform them by constant 1000.
    '''

    return pd.to_datetime(int(timestamp_integer / 1000), utc=True, unit='s')

def transform_to_timestamp_integer(datetime_object):
    '''
    As data points involved milliseconds, we need to transform them by constant 1000.
    '''

    return int(datetime_object.timestamp() * 1000)

def obtain_full_spotdata(start_timestamp,
                         end_timestamp,
                         exchange, symbol, interval = '1h',
                         limit = 1000):

    time_difference = int(convert_to_seconds(interval) * limit * 1000)

    full_data_list = []

    curr_time = start_timestamp + time_difference
    while (curr_time + time_difference < end_timestamp):
        data_list = get_spot(exchange = exchange, symbol = symbol, interval = interval,
                             endTime = curr_time,
                             limit = limit)
        full_data_list = full_data_list + data_list

        time.sleep(0.2)
        curr_time += time_difference

    data_list = get_spot(exchange = exchange, symbol = symbol, interval = interval,
                        startTime = curr_time,
                        endTime = end_timestamp,
                        limit = limit)

    full_data_list = full_data_list + data_list

    return full_data_list

In [4]:
start_date = "2016-01-01"
end_date = "2025-02-24"
interval = "1d"

In [5]:
def fetch_data(ticker_symbol, start_date, end_date, interval):
    # Convert dates to timestamps in milliseconds
    start_timestamp = int(((pd.to_datetime(start_date)).tz_localize('UTC')).timestamp() * 1000)
    end_timestamp = int(((pd.to_datetime(end_date)).tz_localize('UTC')).timestamp() * 1000)

    # Obtain raw data from Binance US
    full_data_list = obtain_full_spotdata(start_timestamp, end_timestamp, binance_exchange, ticker_symbol, interval=interval)

    # Create a DataFrame with the correct column names
    data = pd.DataFrame(full_data_list, columns=SPOT_COLUMNS)
    data['Open time'] = data['Open time'].apply(lambda x: transform_timestamp(int(x)))

    # Remove any duplicate entries based on 'Open time'
    data.drop_duplicates('Open time', keep='first', inplace=True)

    # Add a column to identify the ticker
    data['Ticker'] = ticker_symbol
    return data

In [6]:
all_data = []
for ticker in tqdm(active_usdt_tickers, desc="Fetching crypto data"):
    try:
        cleaned_ticker = ticker.replace('/', '')
        df = fetch_data(cleaned_ticker, start_date, end_date, interval)
        all_data.append(df)
        print(f"Data for {cleaned_ticker} fetched successfully.")
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Concatenate all the DataFrames into one master DataFrame
master_data = pd.concat(all_data, ignore_index=True)

# Save the master DataFrame to a Parquet file
master_data.to_parquet("master_crypto_data.parquet", index=False)
print("Master parquet file saved as 'master_crypto_data.parquet'")

Fetching crypto data:   1%|          | 1/162 [00:00<01:43,  1.56it/s]

Data for BTCUSDT fetched successfully.


Fetching crypto data:   1%|          | 2/162 [00:01<01:36,  1.65it/s]

Data for ETHUSDT fetched successfully.


Fetching crypto data:   2%|▏         | 3/162 [00:01<01:32,  1.71it/s]

...

Data for POLUSDT fetched successfully.


Fetching crypto data:  99%|█████████▉| 161/162 [01:31<00:00,  1.87it/s]

Data for TRUMPUSDT fetched successfully.


Fetching crypto data: 100%|██████████| 162/162 [01:31<00:00,  1.76it/s]

Data for NEIROUSDT fetched successfully.
Master parquet file saved as 'master_crypto_data.parquet'


In [8]:
master_data

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore,Ticker
0,2019-09-23 00:00:00+00:00,9930.13000000,9930.13000000,9930.13000000,9930.13000000,0.00100000,1569283199999,9.93013000,1,0.00000000,0.00000000,0,BTCUSDT
1,2019-09-24 00:00:00+00:00,9637.93000000,9665.05000000,7996.45000000,8519.67000000,202.35079900,1569369599999,1739432.08920509,2345,125.93546400,1075623.60280331,0,BTCUSDT
2,2019-09-25 00:00:00+00:00,8528.12000000,8724.77000000,8247.78000000,8435.28000000,55.41797100,1569455999999,469361.69055629,763,29.05942700,246049.20514584,0,BTCUSDT
3,2019-09-26 00:00:00+00:00,8434.45000000,8453.20000000,7761.87000000,8064.41000000,41.88007900,1569542399999,340923.66601370,1023,19.80884900,161115.44149782,0,BTCUSDT
4,2019-09-27 00:00:00+00:00,8078.75000000,8259.96000000,7858.58000000,8178.73000000,47.14290400,1569628799999,379917.99846542,990,23.90374200,192755.96772991,0,BTCUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66546,2025-02-20 00:00:00+00:00,0.00034998,0.00036500,0.00031545,0.00036500,6698622.00,1740095999999,2359.70839526,31,6060991.00,2154.55577876,0,NEIROUSDT
66547,2025-02-21 00:00:00+00:00,0.00036134,0.00044000,0.00034561,0.00039196,184682301.00,1740182399999,70928.69800741,132,181332829.00,69700.94674665,0,NEIROUSDT
66548,2025-02-22 00:00:00+00:00,0.00036830,0.00044590,0.00035281,0.00043700,7017510.00,1740268799999,2906.92565489,58,6191501.00,2586.24623479,0,NEIROUSDT
66549,2025-02-23 00:00:00+00:00,0.00039728,0.00043262,0.00037002,0.00038312,14115426.00,1740355199999,5717.43299936,33,10311821.00,4228.42642474,0,NEIROUSDT
